# Phase 9.2: FFT Method Comparison

This notebook compares different FFT computation methods:
- NumPy FFT (basic)
- SciPy FFT (Welch method)
- MATLAB FFT (from teammate)

## Setup & Configuration

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import signal
import scipy.fftpack

# Add project to path
project_root = Path('..')
sys.path.insert(0, str(project_root))

try:
    from src.utils import MATLABBridge, MathUtils
    HAS_UTILS = True
except ImportError:
    HAS_UTILS = False
    print("⚠ Utils not available, using basic implementations")

# Configure plotting
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Configuration
OUTPUT_DIR = Path('outputs/fft_comparison')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

SAMPLING_RATE_MATLAB = 4654.545  # Hz (MATLAB)
SAMPLING_RATE_INTERNAL = 10000   # Hz (internal)
FFT_SIZE = 2048
FREQ_MIN = 30
FREQ_MAX = 2000

print(f"Output directory: {OUTPUT_DIR}")

## Generate Sample Data

In [ ]:
# Generate synthetic signal with known frequencies
np.random.seed(42)

# Test frequencies
test_freqs = [60.782, 248.3, 356.67, 477.35]  # Hz
duration = 0.5  # seconds
fs = SAMPLING_RATE_INTERNAL
t = np.arange(0, duration, 1/fs)

# Create test signal
signal_test = np.zeros_like(t)
for freq in test_freqs:
    amplitude = np.random.uniform(0.1, 1.0)
    signal_test += amplitude * np.sin(2 * np.pi * freq * t)

# Add noise
signal_test += 0.1 * np.random.randn(len(t))

print(f"Test signal created:")
print(f"  Duration: {duration} s")
print(f"  Sampling rate: {fs} Hz")
print(f"  Samples: {len(signal_test)}")
print(f"  Known frequencies: {test_freqs}")

## NumPy FFT Method

In [ ]:
# NumPy FFT (basic)
fft_numpy = np.fft.fft(signal_test, n=FFT_SIZE)
mag_numpy = np.abs(fft_numpy) / len(signal_test) * 2  # Normalize
mag_numpy = mag_numpy[:FFT_SIZE//2]  # Keep positive frequencies
freq_numpy = np.fft.fftfreq(FFT_SIZE, 1/fs)[:FFT_SIZE//2]

print(f"NumPy FFT:")
print(f"  Frequency bins: {len(freq_numpy)}")
print(f"  Freq range: [{freq_numpy[0]:.2f}, {freq_numpy[-1]:.2f}] Hz")
print(f"  Max magnitude: {mag_numpy.max():.4f}")
print(f"  Shape: {mag_numpy.shape}")

## SciPy FFT Method (Welch)

In [ ]:
# SciPy Welch method (averaged periodogram)
freq_scipy, pxx_scipy = signal.welch(
    signal_test,
    fs=fs,
    window='hann',
    nperseg=min(FFT_SIZE, len(signal_test)),
    noverlap=None,
    nfft=FFT_SIZE
)

# Filter to target frequency range
mask = (freq_scipy >= FREQ_MIN) & (freq_scipy <= FREQ_MAX)
freq_scipy_filtered = freq_scipy[mask]
mag_scipy = np.sqrt(pxx_scipy[mask])  # Convert power to magnitude

print(f"SciPy Welch FFT:")
print(f"  Frequency bins: {len(freq_scipy_filtered)}")
print(f"  Freq range: [{freq_scipy_filtered[0]:.2f}, {freq_scipy_filtered[-1]:.2f}] Hz")
print(f"  Max magnitude: {mag_scipy.max():.4f}")
print(f"  Shape: {mag_scipy.shape}")

## Method Visualization

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Time series
ax = axes[0, 0]
ax.plot(t[:1000], signal_test[:1000], 'b-', alpha=0.7, linewidth=1)
ax.set_title('Test Signal (Time Domain)', fontweight='bold')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Amplitude')
ax.grid(True, alpha=0.3)

# NumPy FFT
ax = axes[0, 1]
ax.plot(freq_numpy, mag_numpy, 'b-', alpha=0.7, linewidth=1)
for freq in test_freqs:
    if freq <= freq_numpy[-1]:
        ax.axvline(freq, color='r', linestyle='--', alpha=0.5, linewidth=1)
ax.set_title('NumPy FFT', fontweight='bold')
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('Magnitude')
ax.set_xlim([0, 500])
ax.grid(True, alpha=0.3)

# SciPy Welch
ax = axes[1, 0]
ax.plot(freq_scipy_filtered, mag_scipy, 'g-', alpha=0.7, linewidth=1)
for freq in test_freqs:
    if freq <= freq_scipy_filtered[-1]:
        ax.axvline(freq, color='r', linestyle='--', alpha=0.5, linewidth=1)
ax.set_title('SciPy Welch FFT', fontweight='bold')
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('Magnitude')
ax.set_xlim([0, 500])
ax.grid(True, alpha=0.3)

# Overlay comparison
ax = axes[1, 1]
# Normalize for comparison
mag_numpy_norm = mag_numpy / mag_numpy.max()
mag_scipy_norm = mag_scipy / mag_scipy.max()

ax.plot(freq_numpy, mag_numpy_norm, 'b-', alpha=0.7, label='NumPy', linewidth=1.5)
ax.plot(freq_scipy_filtered, mag_scipy_norm, 'g--', alpha=0.7, label='SciPy', linewidth=1.5)
for freq in test_freqs:
    if freq <= 500:
        ax.axvline(freq, color='r', linestyle=':', alpha=0.5, linewidth=1)
ax.set_title('Method Comparison (Normalized)', fontweight='bold')
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('Normalized Magnitude')
ax.set_xlim([0, 500])
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'fft_methods_comparison.png', dpi=150, bbox_inches='tight')
plt.show()
print("✓ Saved: fft_methods_comparison.png")

## Correlation Analysis

In [ ]:
# Compute correlation between methods
from scipy.interpolate import interp1d

# Interpolate to common grid for comparison
common_freqs = np.linspace(30, 500, 200)

f_numpy_interp = interp1d(freq_numpy, mag_numpy, kind='cubic', fill_value='extrapolate')
f_scipy_interp = interp1d(freq_scipy_filtered, mag_scipy, kind='cubic', fill_value='extrapolate')

mag_numpy_common = f_numpy_interp(common_freqs)
mag_scipy_common = f_scipy_interp(common_freqs)

# Normalize
mag_numpy_common = mag_numpy_common / mag_numpy_common.max()
mag_scipy_common = mag_scipy_common / mag_scipy_common.max()

# Compute metrics
correlation = np.corrcoef(mag_numpy_common, mag_scipy_common)[0, 1]
mse = np.mean((mag_numpy_common - mag_scipy_common) ** 2)
mae = np.mean(np.abs(mag_numpy_common - mag_scipy_common))

print("\n=== Correlation Analysis ===")
print(f"Pearson Correlation: {correlation:.6f}")
print(f"Mean Squared Error:  {mse:.6f}")
print(f"Mean Absolute Error: {mae:.6f}")
print("\n✓ Both methods show good agreement" if correlation > 0.9 else "⚠ Methods show differences")

## Frequency Resolution Analysis

In [ ]:
print("\n=== Frequency Resolution ===")

# NumPy resolution
freq_res_numpy = fs / FFT_SIZE
print(f"NumPy FFT:")
print(f"  Frequency resolution: {freq_res_numpy:.4f} Hz")
print(f"  Frequency range: [0, {fs/2:.1f}] Hz")
print(f"  Bins in [30, 2000] Hz: {np.sum((freq_numpy >= 30) & (freq_numpy <= 2000))}")

# SciPy resolution
freq_res_scipy = freq_scipy_filtered[1] - freq_scipy_filtered[0] if len(freq_scipy_filtered) > 1 else 0
print(f"\nSciPy Welch:")
print(f"  Frequency resolution: {freq_res_scipy:.4f} Hz")
print(f"  Frequency range: [{freq_scipy_filtered[0]:.1f}, {freq_scipy_filtered[-1]:.1f}] Hz")
print(f"  Bins in [30, 2000] Hz: {len(freq_scipy_filtered)}")

## Summary Report

In [ ]:
report = f"""
{"="*60}
FFT METHOD COMPARISON REPORT
{"="*60}

METHOD CHARACTERISTICS:

NumPy FFT (Basic):
  ✓ Simple, fast
  ✓ Direct FFT computation
  ✗ No noise reduction
  Pros: Speed, simplicity
  Cons: Spectral leakage

SciPy Welch (Averaged Periodogram):
  ✓ Reduced variance
  ✓ Hanning window (less spectral leakage)
  ✓ Better noise robustness
  Pros: Better SNR, smoother spectra
  Cons: Slightly slower, lower resolution

CORRELATION ANALYSIS:
  Pearson Correlation: {correlation:.6f}
  MSE: {mse:.6f}
  MAE: {mae:.6f}

RECOMMENDATION:
  Use SciPy Welch method for production
  Reason: Better noise robustness and smoother spectra
  Impact: Improved model generalization

FREQUENCY SPECIFICATION:
  Target range: [{FREQ_MIN}, {FREQ_MAX}] Hz
  NumPy bins available: {np.sum((freq_numpy >= FREQ_MIN) & (freq_numpy <= FREQ_MAX))}
  SciPy bins available: {len(freq_scipy_filtered)}

{"="*60}
"""

print(report)

# Save report
with open(OUTPUT_DIR / 'fft_comparison_report.txt', 'w') as f:
    f.write(report)

print("✓ Report saved: fft_comparison_report.txt")

## Next Steps

In [ ]:
print("\n" + "="*60)
print("FFT COMPARISON COMPLETE")
print("="*60)
print("\nOutputs saved to:", OUTPUT_DIR)
print("  - fft_methods_comparison.png")
print("  - fft_comparison_report.txt")
print("\nNext steps:")
print("  1. Review FFT comparison results")
print("  2. Verify frequency resolution is adequate")
print("  3. Confirm method selection (SciPy Welch recommended)")
print("  4. Go to 03_model_experiments.ipynb for model optimization")